In [1]:
!conda install -c conda-forge lat_lon_parser --y

Solving environment: ...working... done

# All requested packages already installed.

Retrieving notices: ...working... done




==> WARNING: A newer version of conda exists. <==
  current version: 4.14.0
  latest version: 23.7.3

Please update conda by running

    $ conda update -n base -c conda-forge conda




In [2]:
!pip install exif

In [3]:
!pip install pillow-heif

In [32]:
import os
import glob
import sys
from lat_lon_parser import parse
import geopandas as gpd
import pandas as pd
import exif
import pillow_heif
import numpy as np

from PIL import Image

In [33]:
# define fucntion to get file exif information
def get_exif(filename):
    image = Image.open(filename)
    image.verify()
    return image.getexif().get_ifd(0x8825)

# define fucntion to get location data from exif
def get_geotagging(exif):
    geo_tagging_info = {}
    # If no file image info return location data as (N:0, W:0)
    if not exif:
        return {'GPSLatitudeRef': 'N',
                'GPSLatitude': '(0, 0, 0)',
                'GPSLongitudeRef': 'W',
                'GPSLongitude': '(0, 0, 0)'}
    else:
        gps_keys = ['GPSVersionID', 'GPSLatitudeRef', 'GPSLatitude', 'GPSLongitudeRef', 'GPSLongitude',
                    'GPSAltitudeRef', 'GPSAltitude', 'GPSTimeStamp', 'GPSSatellites', 'GPSStatus', 'GPSMeasureMode',
                    'GPSDOP', 'GPSSpeedRef', 'GPSSpeed', 'GPSTrackRef', 'GPSTrack', 'GPSImgDirectionRef',
                    'GPSImgDirection', 'GPSMapDatum', 'GPSDestLatitudeRef', 'GPSDestLatitude', 'GPSDestLongitudeRef',
                    'GPSDestLongitude', 'GPSDestBearingRef', 'GPSDestBearing', 'GPSDestDistanceRef', 'GPSDestDistance',
                    'GPSProcessingMethod', 'GPSAreaInformation', 'GPSDateStamp', 'GPSDifferential']

        for k, v in exif.items():
            try:
                geo_tagging_info[gps_keys[k]] = str(v)
            except IndexError:
                pass
        return geo_tagging_info


In [34]:
# set file path to image directory (downlaoded from one drive shared file)
path = "C:/Users/mediwils/Downloads/ImagesFromLeedsLSOAs/"

# Problem with backslash: "\\"
# cerate empty list of dfs
dfs= []
master_df =pd.DataFrame()
paths =os.listdir(path)

# For sub-directory in the full directory
for p in paths:
    for root, dirs, files in os.walk(path+p):
        # Get the name of all files in directory
        for name in files:
            # get jpg files
            if name.endswith(".JPG"):
                file_path = os.path.join(root, name)
                image_info = get_exif(file_path)
                results = get_geotagging(image_info)
                df=pd.DataFrame(columns =['IMGid','Latitude','Longitude'])
                df.loc[0]= [name.rstrip(".JPG"),parse(results['GPSLatitude']),(parse(results['GPSLongitude'])*-1) ]
                df["file_path"] = str(file_path)
                master_df =pd.concat([master_df,df])
            # if not jpg get jpeg files
            elif name.endswith(".jpeg"):
                file_path = os.path.join(root, name)
                image_info = get_exif(file_path)
                results = get_geotagging(image_info)
                df=pd.DataFrame(columns =['IMGid','Latitude','Longitude'])
                df.loc[0]= [name.rstrip(".jpeg"),parse(results['GPSLatitude']),(parse(results['GPSLongitude'])*-1) ]
                df["file_path"] = str(file_path)
                master_df =pd.concat([master_df,df])  
            elif name.endswith(".jpg"):
                file_path = os.path.join(root, name)
                image_info = get_exif(file_path)
                results = get_geotagging(image_info)
                df=pd.DataFrame(columns =['IMGid','Latitude','Longitude'])
                df.loc[0]= [name.rstrip(".jpg"),parse(results['GPSLatitude']),(parse(results['GPSLongitude'])*-1) ]
                df["file_path"] = str(file_path)
                master_df =pd.concat([master_df,df])  
                print(name)

thumbnail_IMG_6380.jpg
20230519_093817.jpg
20230519_093841.jpg
20230519_094110.jpg
20230519_094250.jpg
20230519_094853.jpg
20230519_094902.jpg
20230519_095437.jpg
20230519_095449.jpg
20230519_095657.jpg
20230519_095803.jpg
20230519_095958.jpg
20230519_100224.jpg
20230519_100234.jpg
20230519_100250.jpg
20230519_101855.jpg
20230519_102007.jpg
20230519_102650.jpg
20230519_102710.jpg
20230519_102729.jpg
20230519_102739.jpg
20230519_102755.jpg
20230519_102817.jpg
20230519_103010.jpg
20230519_104728.jpg
20230519_105248.jpg
20230519_105252.jpg
20230519_111821.jpg
20230519_111852.jpg
20230519_112118.jpg
20230519_112319.jpg
20230519_112331.jpg
20230519_112528.jpg
20230519_112532.jpg
20230519_113120.jpg
20230519_113210.jpg
20230519_113212.jpg
20230519_113225.jpg
20230519_113233.jpg
20230519_113243.jpg
20230519_113253.jpg
20230519_113302.jpg
20230519_113748.jpg
20230519_113754.jpg
20230519_114056.jpg
20230519_114635.jpg
20230519_120034.jpg
20230519_120320.jpg
20230519_121458.jpg
20230519_121513.j

In [35]:
name

'20230523_115524.jpg'

In [213]:
master_df.head()

,IMGid,Latitude,Longitude,file_path,LSOA_naming,lsoa_code,IMD_Q,LSOA_name
0,IMG_5123,53.805711,-1.628422,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011287_Q3_SpringValleys,E01011287,Q3,SpringValleys
0,IMG_5124,53.805664,-1.628519,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011287_Q3_SpringValleys,E01011287,Q3,SpringValleys
0,IMG_5125,53.805928,-1.632267,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011287_Q3_SpringValleys,E01011287,Q3,SpringValleys
0,IMG_5126,53.805928,-1.632200,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011287_Q3_SpringValleys,E01011287,Q3,SpringValleys
0,IMG_5127,53.805908,-1.632492,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011287_Q3_SpringValleys,E01011287,Q3,SpringValleys


In [36]:
# Get LSOA quantile and other info from file path
master_df['LSOA_naming'] =master_df['file_path'].str.split('/', expand=True)[5]
master_df['LSOA_naming'] =master_df['LSOA_naming'].str.split("\\", expand=True)[0]
master_df[['lsoa_code', 'IMD_Q','LSOA_name']]= master_df['LSOA_naming'].str.split('_', expand=True)
# master_df['lsoa_code']= master_df['LSOA_naming'].str.split('_', expand=True)[0]
# master_df['IMD_Q']= master_df['LSOA_naming'].str.split('_', expand=True)[1]
# master_df['LSOA_name']= master_df['LSOA_naming'].str.split('_', expand=True)[2]
master_df['LSOA_name'] = master_df['LSOA_name'].str.replace( r"([A-Z])", r" \1").str.strip()

                                                            # If no location info reaplce with Nan
master_df.loc[(master_df['Latitude']==0),'Longitude']=np.NaN
master_df.loc[(master_df['Latitude']==0),'Latitude']=np.NaN

# Convert to geodataframe 
master_gdf =gpd.GeoDataFrame(master_df,geometry=gpd.points_from_xy(master_df.Longitude, master_df.Latitude))
# Set CRS
master_gdf =master_gdf.set_crs("EPSG:4326")

In [37]:
master_df.tail()

,IMGid,Latitude,Longitude,file_path,LSOA_naming,lsoa_code,IMD_Q,LSOA_name
0,20230523_113033,53.798111,-1.528470,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01033021_Q2_UpperAccommodationRd,E01033021,Q2,UpperAccommodationRd
0,20230523_113538,53.797741,-1.522829,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01033021_Q2_UpperAccommodationRd,E01033021,Q2,UpperAccommodationRd
0,20230523_113831,53.797844,-1.520840,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01033021_Q2_UpperAccommodationRd,E01033021,Q2,UpperAccommodationRd
0,20230523_115503,53.795664,-1.516769,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01033021_Q2_UpperAccommodationRd,E01033021,Q2,UpperAccommodationRd
0,20230523_115524,53.795188,-1.517370,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01033021_Q2_UpperAccommodationRd,E01033021,Q2,UpperAccommodationRd


In [38]:
#look at geo-data frame
master_gdf

,IMGid,Latitude,Longitude,file_path,LSOA_naming,lsoa_code,IMD_Q,LSOA_name,geometry
0,IMG_5123,53.805711,-1.628422,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011287_Q3_SpringValleys,E01011287,Q3,SpringValleys,POINT (-1.62842 53.80571)
0,IMG_5124,53.805664,-1.628519,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011287_Q3_SpringValleys,E01011287,Q3,SpringValleys,POINT (-1.62852 53.80566)
0,IMG_5125,53.805928,-1.632267,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011287_Q3_SpringValleys,E01011287,Q3,SpringValleys,POINT (-1.63227 53.80593)
0,IMG_5126,53.805928,-1.632200,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011287_Q3_SpringValleys,E01011287,Q3,SpringValleys,POINT (-1.63220 53.80593)
0,IMG_5127,53.805908,-1.632492,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011287_Q3_SpringValleys,E01011287,Q3,SpringValleys,POINT (-1.63249 53.80591)
...,...,...,...,...,...,...,...,...,...
0,20230523_113033,53.798111,-1.528470,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01033021_Q2_UpperAccommodationRd,E01033021,Q2,UpperAccommodationRd,POINT (-1.52847 53.79811)
0,20230523_113538,53.797741,-1.522829,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01033021_Q2_UpperAccommodationRd,E01033021,Q2,UpperAccommodationRd,POINT (-1.52283 53.79774)
0,20230523_113831,53.797844,-1.520840,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01033021_Q2_UpperAccommodationRd,E01033021,Q2,UpperAccommodationRd,POINT (-1.52084 53.79784)
0,20230523_115503,53.795664,-1.516769,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01033021_Q2_UpperAccommodationRd,E01033021,Q2,UpperAccommodationRd,POINT (-1.51677 53.79566)


In [39]:
AdvertData = pd.read_excel(r"C:\Users\mediwils\Downloads\AdvertData (1).xlsx")

In [40]:
AdvertData_df= AdvertData[["LSOA", "IMGid"]]

In [41]:
AdvertData_df.head

<bound method NDFrame.head of           LSOA                IMGid
0    E01011597         IMG_4652.JPG
1    E01011597         IMG_4653.JPG
2    E01011597         IMG_4657.JPG
3    E01011597         IMG_4658.JPG
4    E01011597         IMG_4659.JPG
..         ...                  ...
921  E01011271  20230516_162732.jpg
922  E01011271  20230516_162732.jpg
923  E01011271  20230516_165621.jpg
924  E01011271  20230516_165621.jpg
925  E01011271  20230516_171251.jpg

[926 rows x 2 columns]>

In [42]:
merged_df = pd.merge(master_df, AdvertData_df, on="IMGid", how = 'left')

In [43]:
merged_df.head

<bound method NDFrame.head of                IMGid   Latitude  Longitude  \
0           IMG_5123  53.805711  -1.628422   
1           IMG_5124  53.805664  -1.628519   
2           IMG_5125  53.805928  -1.632267   
3           IMG_5126  53.805928  -1.632200   
4           IMG_5127  53.805908  -1.632492   
..               ...        ...        ...   
669  20230523_113033  53.798111  -1.528470   
670  20230523_113538  53.797741  -1.522829   
671  20230523_113831  53.797844  -1.520840   
672  20230523_115503  53.795664  -1.516769   
673  20230523_115524  53.795188  -1.517370   

                                             file_path  \
0    C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...   
1    C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...   
2    C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...   
3    C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...   
4    C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...   
..                                                 ...   
669  C:/Use

In [44]:
df= AreaAttributes = pd.read_excel(r"C:\Users\mediwils\Downloads\AreaAttributes.xlsx")

In [45]:
# Look at geo-data frame
master_gdf

,IMGid,Latitude,Longitude,file_path,LSOA_naming,lsoa_code,IMD_Q,LSOA_name,geometry
0,IMG_5123,53.805711,-1.628422,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011287_Q3_SpringValleys,E01011287,Q3,SpringValleys,POINT (-1.62842 53.80571)
0,IMG_5124,53.805664,-1.628519,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011287_Q3_SpringValleys,E01011287,Q3,SpringValleys,POINT (-1.62852 53.80566)
0,IMG_5125,53.805928,-1.632267,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011287_Q3_SpringValleys,E01011287,Q3,SpringValleys,POINT (-1.63227 53.80593)
0,IMG_5126,53.805928,-1.632200,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011287_Q3_SpringValleys,E01011287,Q3,SpringValleys,POINT (-1.63220 53.80593)
0,IMG_5127,53.805908,-1.632492,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01011287_Q3_SpringValleys,E01011287,Q3,SpringValleys,POINT (-1.63249 53.80591)
...,...,...,...,...,...,...,...,...,...
0,20230523_113033,53.798111,-1.528470,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01033021_Q2_UpperAccommodationRd,E01033021,Q2,UpperAccommodationRd,POINT (-1.52847 53.79811)
0,20230523_113538,53.797741,-1.522829,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01033021_Q2_UpperAccommodationRd,E01033021,Q2,UpperAccommodationRd,POINT (-1.52283 53.79774)
0,20230523_113831,53.797844,-1.520840,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01033021_Q2_UpperAccommodationRd,E01033021,Q2,UpperAccommodationRd,POINT (-1.52084 53.79784)
0,20230523_115503,53.795664,-1.516769,C:/Users/mediwils/Downloads/ImagesFromLeedsLSO...,E01033021_Q2_UpperAccommodationRd,E01033021,Q2,UpperAccommodationRd,POINT (-1.51677 53.79566)


In [46]:
unknown_coord = master_gdf.loc[master_gdf['IMGid']=='20230522_103337', ['Latitude','Longitude']]
master_gdf.loc[master_gdf['IMGid']=='20230522_103324', ['Latitude','Longitude']] = unknown_coord

In [47]:
# # svae file info to excel doc
# master_gdf.to_excel("Image_location_info.xlsx",sheet_name='Image_locations')  

# AdvertData =pd.read_excel('/Users/francescapontin/Downloads/AdvertData.xlsx')

In [48]:
# list columns in advert data 
AdvertData.columns

Index(['Rinitials', 'Cinitials', 'Codedate', 'LSOA', 'IMGid', 'IMGdate',
       'ADid', 'ASSETid', 'Long', 'Lat', 'ADtype', 'ADsize', 'ADprodtype',
       'ADprod', 'ADbrand', 'Brandad', 'Admanagement', 'Price£',
       'Pricesourcelink', 'Unit', 'Portion', 'Totalwt', 'Wtsource',
       'Nutritionsource', 'Nutsourcelink', 'Fatdensity', 'Satfatdensity',
       'Sugardensity', 'Sodiumdensity', 'Ekcaldensity', 'Fibredensity',
       'Proteindensity', 'FVN', 'NPMscore', 'NPMstatus', 'ADcompliance'],
      dtype='object')

In [49]:
# remove file extension to get an image name
AdvertData['IMGid'] =AdvertData['IMGid'].str.strip('.JPG').str.strip('.jpg')

# Get a count of Image locaiton data from files that are in the master spreadsheet
master_gdf['IMGid'].isin(AdvertData['IMGid'].str.strip('.JPG').str.strip('.jpg')).value_counts()

IMGid
True     641
False     33
Name: count, dtype: int64

In [50]:
# remove duplicates
AdvertData['duplicate_image_lsoa'] =AdvertData.duplicated(subset=['IMGid','LSOA'], keep=False)

In [51]:
AdvertData.loc[AdvertData['duplicate_image_lsoa']==True,:]

,Rinitials,Cinitials,Codedate,LSOA,IMGid,IMGdate,ADid,ASSETid,Long,Lat,...,Sugardensity,Sodiumdensity,Ekcaldensity,Fibredensity,Proteindensity,FVN,NPMscore,NPMstatus,ADcompliance,duplicate_image_lsoa
5,IW,IW,2023-05-17,E01011597,IMG_4660,2023-05-11,CF_GLMR_06_01,CF_GLMR_06,NaN,NaN,...,2.61,1033.00,312.70,1.3,17.51,0.0,20.0,1.0,1.0,True
6,IW,IW,2023-05-17,E01011597,IMG_4660,2023-05-11,CF_GLMR_06_01,CF_GLMR_06,NaN,NaN,...,4.70,44.00,54.00,0.0,4.10,0.0,0.0,0.0,0.0,True
19,IW,IW,2023-05-17,E01011597,IMG_4651,2023-05-11,CF_GLMR_14_01,CF_GLMR_14,NaN,NaN,...,0.00,4.00,117.00,0.6,2.80,0.0,0.0,0.0,0.0,True
20,IW,IW,2023-05-17,E01011597,IMG_4651,2023-05-11,CF_GLMR_14_01,CF_GLMR_14,NaN,NaN,...,2.50,320.00,72.00,1.5,10.70,0.0,-3.0,0.0,0.0,True
21,IW,IW,2023-05-17,E01011597,IMG_4677,2023-05-11,CF_GLMR_15_01,CF_GLMR_15,NaN,NaN,...,0.15,153.85,192.31,0.0,2.69,0.0,2.0,0.0,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
920,MM,IW,2023-06-20,E01011271,20230516_162732,2023-05-16,GR_FHHS_05_01,GR_FHHS_05,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
921,MM,IW,2023-06-20,E01011271,20230516_162732,2023-05-16,GR_FHHS_05_02,GR_FHHS_05,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
922,MM,IW,2023-06-20,E01011271,20230516_162732,2023-05-16,GR_FHHS_05_03,GR_FHHS_05,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
923,MM,IW,2023-06-20,E01011271,20230516_165621,2023-05-16,GR_FHHS_06_01,GR_FHHS_06,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [52]:
# Add in validation step once data pre-processing complete
#a.merge(master_gdf, left_on=['IMGname','LSOA'], right_on=['IMGname','lsoa_code'], how='left',validate="1:1")

In [53]:
Add_gdf =gpd.GeoDataFrame(AdvertData.merge(master_gdf, left_on=['IMGid','LSOA'], right_on=['IMGid','lsoa_code'], how='left', indicator=True))

In [54]:
# re-format datetime to avoid explore() potting issues
Add_gdf_plot =Add_gdf
Add_gdf_plot['Codedate'] =Add_gdf['Codedate'].astype(str)
Add_gdf_plot['IMGdate'] =Add_gdf['IMGdate'].astype(str)

In [55]:
# For now only plot images with locaiton info
Add_gdf_plot=Add_gdf_plot.dropna(subset=['geometry'])

In [56]:
Add_gdf_plot.to_excel("Advert_image_location_info.xlsx",sheet_name='Image_locations')  



In [57]:
Add_gdf_plot.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 875 entries, 0 to 891
Data columns (total 46 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   Rinitials             875 non-null    object  
 1   Cinitials             875 non-null    object  
 2   Codedate              875 non-null    object  
 3   LSOA                  875 non-null    object  
 4   IMGid                 875 non-null    object  
 5   IMGdate               875 non-null    object  
 6   ADid                  875 non-null    object  
 7   ASSETid               875 non-null    object  
 8   Long                  0 non-null      float64 
 9   Lat                   0 non-null      float64 
 10  ADtype                874 non-null    float64 
 11  ADsize                874 non-null    float64 
 12  ADprodtype            867 non-null    float64 
 13  ADprod                868 non-null    object  
 14  ADbrand               851 non-null    object  
 15  Bra

In [58]:
# Also save as geojson
Add_gdf_plot.drop(columns='_merge').to_file("Advert_image_location_info.geojson", driver='GeoJSON')